In [1]:
import pandas as pd
import numpy as np
import joblib 

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans


#import xgboost as xgb
import lightgbm as lgb
import re


/Users/aminatkhamokova/anaconda3/lib/python3.11/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [2]:
pip list

Package                            Version
---------------------------------- ------------
absl-py                            2.0.0
aiobotocore                        2.7.0
aiohttp                            3.9.0
aioitertools                       0.7.1
aiosignal                          1.2.0
alabaster                          0.7.12
anaconda-anon-usage                0.4.2
anaconda-catalogs                  0.2.0
anaconda-client                    1.12.1
anaconda-cloud-auth                0.1.4
anaconda-navigator                 2.5.0
anaconda-project                   0.11.1
anyio                              3.5.0
appdirs                            1.4.4
applaunchservices                  0.3.0
appnope                            0.1.2
appscript                          1.1.2
archspec                           0.2.1
argon2-cffi                        21.3.0
argon2-cffi-bindings               21.2.0
arrow                              1.2.3
astroid                            2.14.2
a

In [2]:
#downgrade to python 3.11 to run this 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import catboost 
from catboost import CatBoostClassifier
from catboost import CatBoostRegressor



In [41]:
#binary classification
bc = pd.read_csv('dataset/adults_train.csv')
X_bc = bc.iloc[:,:-1]
y_bc = bc.iloc[:,-1]


In [4]:
#regression
reg = pd.read_csv('dataset/insurance_train.csv')
X_reg = reg.iloc[:,:-1]
y_reg = reg.iloc[:,-1]

In [5]:
#multiclass classification
mc = pd.read_csv('dataset/healthcare_train.csv')
X_mc = mc.iloc[:,:-1]
y_mc = mc.iloc[:,-1]

In [6]:
#clustering
clust = pd.read_csv('dataset/customers.csv')
X_clust = clust

In [166]:
#Initialize and train XGBoost for binary classification, sklearn wrapper and booster
model = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss')
model.fit(X_bc, y_bc)
joblib.dump(model, 'models/xgboost-2_0_2_sklearn-binary_classification.joblib')

xgb_params = {
    'objective': 'binary:logistic',  
    'eval_metric': 'logloss',        
    'max_depth': 3,
    'learning_rate': 0.1,
    'random_state': 42
}
dtrain = xgb.DMatrix(X_bc, label=y_bc)
xgb_booster = xgb.train(xgb_params, dtrain)
joblib.dump(xgb_booster, 'models/xgboost-2_0_2-binary_classification.joblib')


['models/xgboost-2_0_2-binary_classification.joblib']

In [167]:
#Initialize and train XGBoost for regression, sklearn wrapper and booster
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=3)
model.fit(X_reg, y_reg)
joblib.dump(model, 'models/xgboost-2_0_2_sklearn-regression.joblib')

xgb_params = {
    'objective': 'reg:squarederror',  
    'eval_metric': 'rmse',             
    'max_depth': 3,
    'learning_rate': 0.1,
    'seed': 42
}
dtrain = xgb.DMatrix(X_reg, label=y_reg)
xgb_booster = xgb.train(xgb_params, dtrain, 100)
joblib.dump(xgb_booster, 'models/xgboost-2_0_2-regression.joblib')



['models/xgboost-2_0_2-regression.joblib']

In [168]:
# Initialize and train XGBoost for multiclass classification, sklearn wrapper and booster
model = xgb.XGBClassifier(objective='multi:softmax', num_class=len(set(y_mc)), eval_metric='mlogloss')
model.fit(X_mc, y_mc)
joblib.dump(model, 'models/xgboost-2_0_2_sklearn-multiclass_classification.joblib')

xgb_params = {
    'objective': 'multi:softmax',  
    'num_class': len(set(y_mc)),                 
    'eval_metric': 'merror',        
    'max_depth': 3,
    'learning_rate': 0.1,
    'seed': 42
}
dtrain = xgb.DMatrix(X_mc, label=y_mc)
xgb_booster = xgb.train(xgb_params, dtrain, 100)
joblib.dump(xgb_booster, 'models/xgboost-2_0_2-multiclass_classification.joblib')


['models/xgboost-2_0_2-multiclass_classification.joblib']

In [25]:
# Initialize and train LightGBM for binary classification, sklearn wrapper and booster
model = lgb.LGBMClassifier(objective='binary',
                         boosting_type='gbdt',
                         num_leaves=31,
                         learning_rate=0.05,
                         feature_fraction=0.9)
model.fit(X_bc, y_bc)
joblib.dump(model, 'models/lightgbm-4_2_0_sklearn-binary_classification.joblib')

train_data = lgb.Dataset(X_bc, label=y_bc)
lgb_params = {
    'objective': 'binary',
    'metric': 'binary_error',  
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    'max_depth': -1
}
bst = lgb.train(lgb_params, train_data, 100)
print(bst.predict(X_bc))
joblib.dump(bst, 'models/lightgbm-4_2_0-binary_classification.joblib')



[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Number of positive: 247, number of negative: 553
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001938 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 459
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 62
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.308750 -> initscore=-0.805970
[LightGBM] [Info] Start training from score -0.805970
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positiv

['models/lightgbm-4_2_0-binary_classification.joblib']

In [232]:
# Initialize and train LightGBM for regression, sklearn wrapper and booster
model = lgb.LGBMRegressor(objective='regression',
                        boosting_type='gbdt',
                        num_leaves=31,
                        learning_rate=0.05,
                        feature_fraction=0.9)
model.fit(X_reg, y_reg)
joblib.dump(model, 'models/lightgbm-4_2_0_sklearn-regression.joblib')

train_data = lgb.Dataset(X_reg, label=y_reg)
lgb_params = {
    'objective': 'regression',  
    'metric': 'rmse',            
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}
bst = lgb.train(lgb_params, train_data, 100)
joblib.dump(bst, 'models/lightgbm-4_2_0-regression.joblib')


[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000370 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 315
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 7
[LightGBM] [Info] Start training from score 13173.332294
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further spl

['models/lightgbm-4_2_0-regression.joblib']

In [233]:
# Initialize and train LightGBM for multiclass classification, sklearn wrapper and booster
model = lgb.LGBMClassifier(objective='multiclass',
                         num_class=3,  # Number of classes in your dataset
                         boosting_type='gbdt',
                         num_leaves=31,
                         learning_rate=0.05,
                         feature_fraction=0.9)
model.fit(X_mc, y_mc)
joblib.dump(model, 'models/lightgbm-4_2_0_sklearn-multiclass_classification.joblib')

train_data = lgb.Dataset(X_mc, label=y_mc)
lgb_params = {
    'objective': 'multiclass',  
    'num_class': len(set(y_mc)),              
    'metric': 'multi_logloss',   
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}
bst = lgb.train(lgb_params, train_data, 100)
joblib.dump(bst, 'models/lightgbm-4_2_0-multiclass_classification.joblib')


[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000438 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 596
[LightGBM] [Info] Number of data points in the train set: 8000, number of used features: 10
[LightGBM] [Info] Start training from score -1.068932
[LightGBM] [Info] Start training from score -1.114742
[LightGBM] [Info] Start training from score -1.112838
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines


['models/lightgbm-4_2_0-multiclass_classification.joblib']

In [13]:
# Initialize and train sklearn for binary classification
model = LogisticRegression(solver='liblinear')
model.fit(X_bc, y_bc)
joblib.dump(model, 'models/scikit-learn-1_3_2-binary_classification.joblib')

['models/scikit-learn-1_3_2-binary_classification.joblib']

In [14]:
# Initialize and train sklearn for regression
model = LinearRegression()
model.fit(X_reg, y_reg)
joblib.dump(model, 'models/scikit-learn-1_3_2-regression.joblib')

['models/scikit-learn-1_3_2-regression.joblib']

In [51]:
# Initialize and train sklearn for multiclass classification
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_mc, y_mc)
joblib.dump(model, 'models/scikit-learn-1_3_2-multiclass_classification.joblib')

['models/scikit-learn-1_3_2-multiclass_classification.joblib']

In [81]:
# Initialize and train sklearn for clustering
model = KMeans(n_clusters=5, random_state=42)
model.fit(X_clust)
joblib.dump(model, 'models/scikit-learn-1_3_2-clustering.joblib')

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


['models/scikit-learn-1_3_2-clustering.joblib']

In [17]:
y_bc

0        0.0
1        0.0
2        1.0
3        1.0
4        0.0
        ... 
38995    0.0
38996    0.0
38997    0.0
38998    0.0
38999    0.0
Name: class, Length: 39000, dtype: float64

In [27]:
# Initialize and train tensorflow model for binary classification
model = Sequential()
model.add(Dense(32, input_dim=X_bc.shape[1], activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_bc, y_bc, epochs=10, batch_size=32)
#joblib.dump(model, 'models/tensorflow-2_15_0-binary_classification.joblib')
model.save('models/tensorflow-2_15_0-binary_classification.h5')
model.save('models/tensorflow-2_15_0-binary_classification.keras')

Epoch 1/10
1219/1219 [==============================] - 0s 286us/step - loss: 2.1922 - accuracy: 0.7762
Epoch 2/10
1219/1219 [==============================] - 0s 279us/step - loss: 1.4163 - accuracy: 0.7993
Epoch 3/10
1219/1219 [==============================] - 0s 281us/step - loss: 1.6981 - accuracy: 0.8082
Epoch 4/10
1219/1219 [==============================] - 0s 278us/step - loss: 1.3253 - accuracy: 0.8091
Epoch 5/10
1219/1219 [==============================] - 0s 281us/step - loss: 1.3250 - accuracy: 0.8060
Epoch 6/10
1219/1219 [==============================] - 0s 291us/step - loss: 1.0642 - accuracy: 0.8093
Epoch 7/10
1219/1219 [==============================] - 0s 283us/step - loss: 1.2226 - accuracy: 0.8108
Epoch 8/10
1219/1219 [==============================] - 0s 281us/step - loss: 1.8657 - accuracy: 0.8092
Epoch 9/10
1219/1219 [==============================] - 0s 284us/step - loss: 1.1292 - accuracy: 0.8131
Epoch 10/10
1219/1219 [==============================] - 0s 282u

/Users/aminatkhamokova/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [13]:
# Initialize and train tensorflow model for regression
model = Sequential()
model.add(Dense(32, input_dim=X_reg.shape[1], activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_reg, y_reg, epochs=10, batch_size=32)
#joblib.dump(model, 'models/tensorflow-2_15_0-regression.joblib')
model.save('models/tensorflow-2_15_0-regression.h5')
model.save('models/tensorflow-2_15_0-regression.keras')


Epoch 1/10
25/25 [==============================] - 0s 394us/step - loss: 317197248.0000
Epoch 2/10
25/25 [==============================] - 0s 354us/step - loss: 316819424.0000
Epoch 3/10
25/25 [==============================] - 0s 336us/step - loss: 316419904.0000
Epoch 4/10
25/25 [==============================] - 0s 331us/step - loss: 315993568.0000
Epoch 5/10
25/25 [==============================] - 0s 314us/step - loss: 315530208.0000
Epoch 6/10
25/25 [==============================] - 0s 287us/step - loss: 314998912.0000
Epoch 7/10
25/25 [==============================] - 0s 306us/step - loss: 314416512.0000
Epoch 8/10
25/25 [==============================] - 0s 294us/step - loss: 313744256.0000
Epoch 9/10
25/25 [==============================] - 0s 305us/step - loss: 313003776.0000
Epoch 10/10
25/25 [==============================] - 0s 278us/step - loss: 312177312.0000


/Users/aminatkhamokova/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [14]:
# Initialize and train tensorflow model for multiclass classification 
model = Sequential()
model.add(Dense(32, input_dim=X_mc.shape[1], activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(3, activation='softmax'))  # Number of classes is set to 3
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_mc, y_mc, epochs=10, batch_size=32)
#joblib.dump(model, 'models/tensorflow-2_15_0-multiclass_classification.joblib')
model.save('models/tensorflow-2_15_0-multiclass_classification.h5')
model.save('models/tensorflow-2_15_0-multiclass_classification.keras')


Epoch 1/10
250/250 [==============================] - 0s 355us/step - loss: 240.1648 - accuracy: 0.3300
Epoch 2/10
250/250 [==============================] - 0s 352us/step - loss: 46.2621 - accuracy: 0.3340
Epoch 3/10
250/250 [==============================] - 0s 346us/step - loss: 43.4339 - accuracy: 0.3350
Epoch 4/10
250/250 [==============================] - 0s 338us/step - loss: 52.1187 - accuracy: 0.3349
Epoch 5/10
250/250 [==============================] - 0s 336us/step - loss: 35.7486 - accuracy: 0.3392
Epoch 6/10
250/250 [==============================] - 0s 338us/step - loss: 45.2107 - accuracy: 0.3204
Epoch 7/10
250/250 [==============================] - 0s 322us/step - loss: 48.7509 - accuracy: 0.3326
Epoch 8/10
250/250 [==============================] - 0s 336us/step - loss: 46.7899 - accuracy: 0.3339
Epoch 9/10
250/250 [==============================] - 0s 334us/step - loss: 60.1546 - accuracy: 0.3296
Epoch 10/10
250/250 [==============================] - 0s 424us/step - l

/Users/aminatkhamokova/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [9]:
# Initialize and train catboost model for binary classification, sklearn and booster
model = CatBoostClassifier(iterations=500, depth=10, learning_rate=0.05, loss_function='Logloss')
model.fit(X_bc, y_bc, early_stopping_rounds=50, verbose=100)
joblib.dump(model, 'models/catboost-1_2_2_sklearn-binary_classification.joblib')

train_pool = catboost.Pool(X_bc, label=y_bc)
catboost_params = {
    'iterations': 100,
    'learning_rate': 0.1,
    'depth': 6,
    'loss_function': 'Logloss',  
    'verbose': 0
}
catboost_model = CatBoostClassifier(**catboost_params)
catboost_model.fit(train_pool)
joblib.dump(catboost_model, 'models/catboost-1_2_2-binary_classification.joblib')



0:	learn: 0.6680273	total: 13.5ms	remaining: 6.74s
100:	learn: 0.1287385	total: 942ms	remaining: 3.72s
200:	learn: 0.0516764	total: 1.9s	remaining: 2.83s
300:	learn: 0.0282615	total: 2.85s	remaining: 1.88s
400:	learn: 0.0186566	total: 3.79s	remaining: 936ms
499:	learn: 0.0136905	total: 4.73s	remaining: 0us


['models/catboost-1_2_2-binary_classification.joblib']

In [10]:
# Initialize and train catboost model for regression, sklearn and booster
model = CatBoostRegressor(iterations=500, depth=10, learning_rate=0.05, loss_function='RMSE')
model.fit(X_reg, y_reg, early_stopping_rounds=50, verbose=100)
joblib.dump(model, 'models/catboost-1_2_2_sklearn-regression.joblib')

train_pool = catboost.Pool(X_reg, label=y_reg)
catboost_params = {
    'iterations': 100,
    'learning_rate': 0.1,
    'depth': 6,
    'loss_function': 'RMSE',  
    'verbose': 0
}
catboost_model = CatBoostRegressor(**catboost_params)
catboost_model.fit(train_pool)
joblib.dump(catboost_model, 'models/catboost-1_2_2-regression.joblib')



0:	learn: 11593.9832656	total: 3.8ms	remaining: 1.9s
100:	learn: 3574.9122511	total: 145ms	remaining: 574ms
200:	learn: 2789.0024317	total: 327ms	remaining: 486ms
300:	learn: 2323.7274404	total: 528ms	remaining: 349ms
400:	learn: 1962.7708051	total: 728ms	remaining: 180ms
499:	learn: 1700.3736051	total: 920ms	remaining: 0us


['models/catboost-1_2_2-regression.joblib']

In [44]:
# Initialize and train catboost model for multiclass classification, sklearn and booster
model = CatBoostClassifier(iterations=500, depth=10, learning_rate=0.05, loss_function='MultiClass')
model.fit(X_mc, y_mc, early_stopping_rounds=50, verbose=100)
joblib.dump(model, 'models/catboost-1_2_2_sklearn-multiclass_classification.joblib')

train_pool = catboost.Pool(X_mc, label=y_mc)
catboost_params = {
    'iterations': 100,
    'learning_rate': 0.1,
    'depth': 6,
    'loss_function': 'MultiClass',  
    'verbose': 0
}
catboost_model = CatBoostClassifier(**catboost_params)
catboost_model.fit(train_pool)
joblib.dump(catboost_model, 'models/catboost-1_2_2-multiclass_classification.joblib')

0:	learn: 1.0947772	total: 14.3ms	remaining: 7.13s
100:	learn: 0.8642666	total: 779ms	remaining: 3.08s
200:	learn: 0.7124993	total: 1.55s	remaining: 2.3s
300:	learn: 0.6120539	total: 2.31s	remaining: 1.53s
400:	learn: 0.5325977	total: 3.07s	remaining: 759ms
499:	learn: 0.4668946	total: 3.83s	remaining: 0us


['models/catboost-1_2_2-multiclass_classification.joblib']

In [28]:
import torch
import torch.nn as nn
import torch.optim as optim

In [56]:
# Initialize and train PyTorch model for binary classification
X_train = torch.Tensor(X_bc.values)
y_train = torch.Tensor(y_bc)
# Define a simple logistic regression model
model = nn.Sequential(
    nn.Linear(X_train.shape[1], 1),
    nn.Sigmoid()
)
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
num_epochs = 100
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train.view(-1, 1))
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

#torch.save(model.state_dict(), 'models/torch-2_3_0-binary_classification.joblib')
torch.save(model,'models/torch-2_2_1-binary_classification.pth')

Epoch [100/100], Loss: 10.1416


In [57]:
model_name = "models/torch-2_2_1-binary_classification.pth"
model = torch.load(model_name)
df_test = pd.read_csv("dataset/adults_test.csv")
tensor = torch.Tensor(df_test.drop(columns=["class"]).values)
output = model(tensor)
predicted_classes = (output > 0.5).int()  # Use threshold 0.5 for binary classification


In [14]:
# Initialize and train PyTorch model for regression
X_train = torch.FloatTensor(X_reg.values)
y_train = torch.FloatTensor(y_reg)
# Define a simple regression model
model = nn.Sequential(
    nn.Linear(X_train.shape[1], 1)
)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
num_epochs = 1000
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train.view(-1, 1))
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

#torch.save(model.state_dict(), 'models/torch-2_3_0-regression.joblib')
torch.save(model,'models/torch-2_2_1-regression.pth')

Epoch [100/1000], Loss: 317471936.0000
Epoch [200/1000], Loss: 317450368.0000
Epoch [300/1000], Loss: 317428832.0000
Epoch [400/1000], Loss: 317407328.0000
Epoch [500/1000], Loss: 317385728.0000
Epoch [600/1000], Loss: 317364224.0000
Epoch [700/1000], Loss: 317342688.0000
Epoch [800/1000], Loss: 317321120.0000
Epoch [900/1000], Loss: 317299616.0000
Epoch [1000/1000], Loss: 317278048.0000


In [15]:
# Initialize and train PyTorch model for multiclass classification
X_train = torch.FloatTensor(X_mc.values)
y_train = torch.LongTensor(y_mc)
# Define a simple multiclass classification model
model = nn.Sequential(
    nn.Linear(X_train.shape[1], 3)  # 3 output neurons for three classes
)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
num_epochs = 1000
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

#torch.save(model.state_dict(), 'models/torch-2_3_0-multiclass_classification.joblib')
torch.save(model,'models/torch-2_2_1-multiclass_classification.pth')

Epoch [100/1000], Loss: 2663953.2500
Epoch [200/1000], Loss: 966165.8750
Epoch [300/1000], Loss: 2535061.2500
Epoch [400/1000], Loss: 4042209.0000
Epoch [500/1000], Loss: 2311546.2500
Epoch [600/1000], Loss: 853570.1875
Epoch [700/1000], Loss: 2422496.7500
Epoch [800/1000], Loss: 3576131.7500
Epoch [900/1000], Loss: 1846691.8750
Epoch [1000/1000], Loss: 642834.0000


In [13]:
lightgbm_bc = joblib.load('models/lightgbm-4_2_0-binary_classification.joblib')
lightgbm_reg = joblib.load('models/lightgbm-4_2_0-regression.joblib')
lightgbm_mc = joblib.load('models/lightgbm-4_2_0-multiclass_classification.joblib')

In [203]:
xgb_bc = joblib.load('models/xgboost-2_0_2-binary_classification.joblib')
xgb_reg = joblib.load('models/xgboost-2_0_2-regression.joblib')
xgb_mc = joblib.load('models/xgboost-2_0_2-multiclass_classification.joblib')

In [6]:
cat_bc = joblib.load('models/catboost-1_2_2-binary_classification.joblib')
cat_reg = joblib.load('models/catboost-1_2_2-regression.joblib')
cat_mc = joblib.load('models/catboost-1_2_2-multiclass_classification.joblib')

In [7]:
from sklearn.utils.validation import check_is_fitted
from sklearn.base import BaseEstimator, ClassifierMixin


class LGBMBoosterWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, booster=None):
        self.booster_params = booster.params
        self.booster = booster

    def fit(self, X, y):
        self.booster = lgb.train(self.booster_params, lgb.Dataset(X, label=y))
        return self

    def predict(self, X):
        check_is_fitted(self, 'booster')
        preds = self.booster.predict(X)
        pred_class = (preds > 0.5).astype("int")
        return pred_class
    
    def predict_proba(self, X):
        check_is_fitted(self, 'booster')
        return self.booster.predict(X)

class CatBoosterWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, booster=None):
        self.booster_params = booster.get_params()
        self.booster = booster
        self.is_classifier = isinstance(booster, CatBoostClassifier)

    def fit(self, X, y):
        self.booster = CatBoostClassifier(**self.booster_params) if self.is_classifier else CatBoostRegressor(**self.booster_params)
        self.booster.fit(catboost.Pool(X, label = y))
        return self

    def predict(self, X):
        check_is_fitted(self, 'booster')
        return self.booster.predict(X)   
    
    def predict_proba(self, X):
        check_is_fitted(self, 'booster')
        if self.is_classifier:
            return self.booster.predict_proba(X)
        else:
            raise AttributeError("predict_proba is only available for classifiers.")

"""

class XGBBoosterWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, booster=None):
        self.booster_params = booster.get_dump()[0]
        self.booster = booster

    def fit(self, X, y):
        params = self.booster_params.copy() if self.booster_params is not None else {}
        self.booster = xgb.train(params, xgb.DMatrix(X, label=y))
        return self

    def predict(self, X):
        check_is_fitted(self, 'booster')
        return self.booster.predict(xgb.DMatrix(X))
    
"""    


"\n\nclass XGBBoosterWrapper(BaseEstimator, ClassifierMixin):\n    def __init__(self, booster=None):\n        self.booster_params = booster.get_dump()[0]\n        self.booster = booster\n\n    def fit(self, X, y):\n        params = self.booster_params.copy() if self.booster_params is not None else {}\n        self.booster = xgb.train(params, xgb.DMatrix(X, label=y))\n        return self\n\n    def predict(self, X):\n        check_is_fitted(self, 'booster')\n        return self.booster.predict(xgb.DMatrix(X))\n    \n"

In [39]:
lbc = LGBMBoosterWrapper(lightgbm_bc)
lreg = LGBMBoosterWrapper(lightgbm_reg)
lmc = LGBMBoosterWrapper(lightgbm_mc)

In [12]:
cbc = CatBoosterWrapper(cat_bc)
creg = CatBoosterWrapper(cat_reg)
cmc = CatBoosterWrapper(cat_mc)